Preprocess the data Peter provided to put it into a ready to analyze form.  save it to the file noted below

In [15]:
output_filename = 'data/yields.parquet'

In [2]:
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm

In [3]:
parquet_file = '~/Downloads/merged_yield_df_area_planeted_harvested_winter_wheat_marked.parquet'
raw0 = pq.read_table(parquet_file).to_pandas()
raw0['Acres Harvested'] = raw0['Acres Harvested'].str.replace(',', '').astype(float)
raw0['Acres Planted'] = raw0['Acres Planted'].str.replace(',', '').astype(float)

#rename columns called County as CountyName
raw0.rename(columns={'County':'CountyName'}, inplace=True)
# new column called County that is the concatenation of State and CountyName
raw0['County'] = raw0['State'] + '/' + raw0['CountyName']
# drop the columns State and CountyName
raw0.drop(columns=['State','CountyName'], inplace=True)
# drop county_name, county_number, state_county_code, state_code
raw0.drop(columns=['qa.landsat','x','y','county_name','county_number','state_county_code','state_code'], inplace=True)



print('raw0: {:,} rows'.format(len(raw0)))


raw0: 24,395,812 rows


In [4]:
# for each year, keep only rows where winter wheat is TRUE
l = []
for y in range(2013,2023):
    mask = (raw0['year'] == y) & (raw0['is_winter_wheat_'+str(y)])
    l.append(raw0[mask])
raw1 = pd.concat(l).sort_values('time')
raw1 = raw0.drop(columns=[c for c in raw0.columns if 'is_winter_wheat' in c])
print('raw1: {:,} rows'.format(len(raw1)))

raw1: 24,395,812 rows


In [5]:
drop_list = ['lat_x', 'lon_x', 'ndvi.sentinel2','qa.sentinel2','lat_y','lon_y','point']
raw2 = raw1.drop(columns=drop_list)

In [6]:
raw3 = raw2.drop(raw2[raw2['ndvi.landsat'] > 1.0].index)
print('raw3: {:,} rows'.format(len(raw3)))
raw4 = raw3.dropna(subset=['ndvi.landsat'])
print('raw4: {:,} rows'.format(len(raw4)))
raw5 = raw4.dropna(subset=['Yield (Bu/Acre)'])
print('raw5: {:,} rows'.format(len(raw5)))

raw3: 24,392,289 rows
raw4: 20,744,556 rows
raw5: 15,888,970 rows


In [7]:
raw5.head()

,time,ndvi.landsat,year,Yield (Bu/Acre),Acres Planted,Acres Harvested,County
0,2018-03-01,0.337683,2018,20.5,42600.0,21000.0,TEXAS/COLEMAN
1,2018-03-02,0.337683,2018,20.5,42600.0,21000.0,TEXAS/COLEMAN
2,2018-03-03,0.337683,2018,20.5,42600.0,21000.0,TEXAS/COLEMAN
19,2018-03-20,0.364731,2018,20.5,42600.0,21000.0,TEXAS/COLEMAN
20,2018-03-21,0.364731,2018,20.5,42600.0,21000.0,TEXAS/COLEMAN


In [9]:

raw6 = raw5[['time','year', 'County','ndvi.landsat', 'Yield (Bu/Acre)','Acres Planted', 'Acres Harvested']]
raw6.head()

,time,year,County,ndvi.landsat,Yield (Bu/Acre),Acres Planted,Acres Harvested
0,2018-03-01,2018,TEXAS/COLEMAN,0.337683,20.5,42600.0,21000.0
1,2018-03-02,2018,TEXAS/COLEMAN,0.337683,20.5,42600.0,21000.0
2,2018-03-03,2018,TEXAS/COLEMAN,0.337683,20.5,42600.0,21000.0
19,2018-03-20,2018,TEXAS/COLEMAN,0.364731,20.5,42600.0,21000.0
20,2018-03-21,2018,TEXAS/COLEMAN,0.364731,20.5,42600.0,21000.0


In [16]:
# save raw6 to disk
raw6.to_parquet(output_filename)

In [18]:
# print the number of unique Counties by year
for y in range(2013,2023):
    mask = (raw6['year'] == y)
    print('Number of unique counties in {}: {:,}'.format(y, len(raw6[mask]['County'].unique())))
    
print('Number of unique counties: {:,}'.format(len(raw6['County'].unique())))

Number of unique counties in 2013: 542
Number of unique counties in 2014: 578
Number of unique counties in 2015: 562
Number of unique counties in 2016: 541
Number of unique counties in 2017: 480
Number of unique counties in 2018: 454
Number of unique counties in 2019: 445
Number of unique counties in 2020: 521
Number of unique counties in 2021: 500
Number of unique counties in 2022: 471
Number of unique counties: 737
